In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load the FER2013 dataset
data = pd.read_csv('/content/drive/MyDrive/fer2013.csv')

# Extract pixel values and labels
pixels = data['pixels'].values
labels = data['emotion'].values

# Convert pixel values to arrays and normalize them
pixels = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in pixels])
pixels = pixels / 255.0  # Normalize to the range [0, 1]
pixels = pixels.reshape(-1, 48, 48, 1)

# Convert labels to categorical format
labels = to_categorical(labels, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pixels, labels, test_size=0.2, random_state=42)

# Normalize test data
X_test = X_test / 255.0

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Calculate class weights manually
class_labels = np.unique(labels.argmax(axis=1))
class_weights = compute_class_weight('balanced', class_labels, labels.argmax(axis=1))
class_weight_dict = dict(zip(class_labels, class_weights))

# Learning Rate Scheduler
def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * np.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with augmented data, class weights, and callbacks
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_test, y_test),
    epochs=50,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler, early_stopping]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Confusion Matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1', '2', '3', '4', '5', '6'], yticklabels=['0', '1', '2', '3', '4', '5', '6'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
# Save the model (optional)
model.save('emotion_detection_model.h5')



TypeError: ignored

In [ ]:
from google.colab import files
import cv2
import numpy as np
from keras.models import load_model

# Upload an image file to Colab
uploaded = files.upload()

# Load the pre-trained emotion detection model
model = load_model('emotion_detection_model.h5')

# Map the emotion label to its corresponding emotion
emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# Read and preprocess the uploaded image
file_name = list(uploaded.keys())[0]
img = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)

# Normalize pixel values to the range [0, 1]
normalized_img = img / 255.0

# Resize the image to match the input shape of the model
normalized_img = cv2.resize(normalized_img, (48, 48))

# Reshape the image to match the input shape of the model
input_image = normalized_img.reshape(1, 48, 48, 1)

# Make predictions using the model
predictions = model.predict(input_image)
emotion_label = np.argmax(predictions)
predicted_emotion = emotion_mapping[emotion_label]

print(f'Predicted Emotion: {predicted_emotion}')



In [ ]:
import matplotlib.pyplot as plt

# Get emotion labels and corresponding probabilities
emotion_labels = list(emotion_mapping.values())
probabilities = predictions.flatten()

# Plot the bar graph
plt.figure(figsize=(10, 6))
plt.bar(emotion_labels, probabilities, color='blue')
plt.xlabel('Emotion')
plt.ylabel('Probability')
plt.title('Predicted Emotion Probabilities')
plt.ylim([0, 1])  # Set y-axis range between 0 and 1 for probability interpretation
plt.show()


In [ ]:
from keras.models import load_model

# Load the pre-trained emotion detection model
model = load_model('emotion_detection_model.h5')

